In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Esophageal_Cancer/GSE106817'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Integrated extracellular microRNA profiling for ovarian cancer screening"
!Series_summary	"A serum miRNA combination could be a powerful classifier for the detEsophageal Cancertion of patients with ovarian cancer."
!Series_overall_design	"Serum microRNA profiles of 4046 woman samples, which consist of 333 of ovarian cancers, 66 of borderline ovarian tumors, 29 of benign ovarian diseases, 2759 of non-cancer controls, and 859 of the other solid cancers."
Sample Characteristics Dictionary:
{0: ['tumor stage (figo): NA', 'tumor stage (figo): 1C', 'tumor stage (figo): 1A', 'tumor stage (figo): 2C', 'tumor stage (figo): 3A', 'tumor stage (figo): 1B', 'tumor stage (figo): 3B', 'tumor stage (figo): 2B', 'tumor stage (figo): 3C', 'tumor stage (figo): XX', 'tumor stage (figo): 4', 'tumor stage (figo): 2A', 'tumor stage (figo): 4A', 'tumor stage (figo): 3', 'tumor stage (figo): 4B'], 1: ['age: uncertain', 'age: 52', 'age: 60', 'age: 57', 'age: 59', 'age: 45'

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Checking if dataset likely contains gene expression data
series_title = "Integrated extracellular microRNA profiling for ovarian cancer screening"
if "miRNA" in series_title or "methylation" in series_title:
    is_gene_available = False
else:
    is_gene_available = True

# Analyzing sample characteristics for 'Esophageal_Cancer', 'age', and 'gender'
sample_characteristics = {
    0: ['tumor stage (figo): NA', 'tumor stage (figo): 1C', 'tumor stage (figo): 1A', 'tumor stage (figo): 2C', 'tumor stage (figo): 3A', 'tumor stage (figo): 1B', 'tumor stage (figo): 3B', 'tumor stage (figo): 2B', 'tumor stage (figo): 3C', 'tumor stage (figo): XX', 'tumor stage (figo): 4', 'tumor stage (figo): 2A', 'tumor stage (figo): 4A', 'tumor stage (figo): 3', 'tumor stage (figo): 4B'],
    1: ['age: uncertain', 'age: 52', 'age: 60', 'age: 57', 'age: 59', 'age: 45', 'age: 63', 'age: 54', 'age: 68', 'age: 48', 'age: 71', 'age: 64', 'age: 58', 'age: 55', 'age: 42', 'age: 46', 'age: 41', 'age: 85', 'age: 51', 'age: 65', 'age: 49', 'age: 39', 'age: 72', 'age: 32', 'age: 77', 'age: 47', 'age: 40', 'age: 73', 'age: 56', 'age: 16']
}

# Data Availability
# 'Esophageal_Cancer' not found in background or sample characteristics -> trait_row = None
trait_row = None

# Checking for 'age' variable
unique_age_values = set([v.split(': ')[1] for v in sample_characteristics[1]])
if len(unique_age_values) > 1 and 'uncertain' not in unique_age_values:
    age_row = 1
else:
    age_row = None

# 'gender' data not found in the sample characteristics
gender_row = None

# Data Type Conversion functions

def convert_trait(value):
    value = value.split(':')[1].strip()
    return None

def convert_age(value):
    value = value.split(':')[1].strip()
    if value.isdigit():
        return int(value)
    else:
        return None

def convert_gender(value):
    value = value.split(':')[1].strip().lower()
    if value == 'male':
        return 1
    elif value == 'female':
        return 0
    else:
        return None

# Save cohort information
save_cohort_info('GSE106817', './preprocessed/Esophageal_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Esophageal_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Esophageal_Cancer/trait_data/GSE106817.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
